# Evaluating All Models Using GPT-4 as an A/B Tester

## Introduction
This notebook contains code to evaluate all A/B tests across all prompts for a benchmark. The code reads in a list of unevaluated AB tests with prompts and responses from a database, runs parallel A/B tests with GPT-4 as a judge, and writes the results back to the database.

## Setup
Before running the code, make sure to install the necessary dependencies by setting up the Poetry environment. You will also need to set up a database connection by filling in the appropriate values in the cells below. You will also need to provide up an OpenAI API key and organization ID (if appropriate).

Additionally, specify the benchmark and evaluating model to use for evaluation. The benchmark is the name of the benchmark to evaluate. The evaluating model is the name of the model to use as a judge for the A/B tests. The evaluating model must be a model that is already in the database.

In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY']="YOUR_API_KEY"
os.environ['OPENAI_ORGANIZATION']="YOUR_ORG_ID"

In [ ]:
os.environ['RDS_HOSTNAME']="YOUR_RDS_DB_HOSTNAME"
os.environ['RDS_DB_NAME']="YOUR_RDS_DB_NAME"
os.environ['RDS_USERNAME']="YOUR_RDS_USERNAME"
os.environ['RDS_PASSWORD']="YOUR_RDS_PASSWORD"

In [ ]:
BENCHMARK = "v1"
EVALUATING_MODEL = "gpt-4-0613"

## Code Structure
The code is split into several sections:

### Imports
The first section imports the necessary modules and functions.

### Data Loading
The second section loads the unevaluated AB test prompts from the database and converts them to requests for the OpenAI API.

### Model Evaluation
The third section evaluates the AB test prompts using the OpenAI cookbook.

### Data Writing
The fourth section parses the evaluation results and writes them back to the database.

## Usage
To use this code, simply run the notebook from start to finish. The results will be written back to the database automatically.

In [ ]:
from db.db_utils import get_all_unevaluated_ab_test_prompts_for_benchmark, insert_evals_by_model_into_db
from src.parse import db_game_to_dict, construct_request, parse_result_to_eval_entry, games_list_to_lookup
from src.utils import read_jsonl, write_jsonl

## Data Loading

In [ ]:
incomplete_game_rows = await get_all_unevaluated_ab_test_prompts_for_benchmark(benchmark_name=BENCHMARK, eval_model_name=EVALUATING_MODEL)
print(f"{len(incomplete_game_rows)} incomplete games found")
if len(incomplete_game_rows) > 0:
    print(f"Example: {incomplete_game_rows[0]}")

In [ ]:
games = [db_game_to_dict(row) for row in incomplete_game_rows]
print(f"{len(games)} games parsed")
if len(games) > 0:
    print(f"Example: {games[0]}")

In [ ]:
requests = [construct_request(game) for game in games]
write_jsonl(requests, 'requests.jsonl')
print(f"{len(requests)} requests written to file")
if len(requests) > 0:
    print(f"Example: {requests[0]}")

## Model Evaluation

In [ ]:
!python ../src/api_request_parallel_processor.py \
  --requests_filepath requests.jsonl \
  --save_filepath requests_results.jsonl \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 200 \
  --max_tokens_per_minute 40000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

## Data Writing

In [ ]:
results, failed_lines = read_jsonl('requests_results.jsonl')

if len(failed_lines):
    print(f"{len(failed_lines)} failed lines read from file")
    print(f"Example: {failed_lines[0]}")

print(f"{len(results)} results read from file")
if len(results) > 0:
    print(f"Example: {results[0]}")

In [ ]:
games_lookup = games_list_to_lookup(games)

eval_entries = [parse_result_to_eval_entry(result, games_lookup) for result in results]
print(f"{len(eval_entries)} eval entries created")
if len(eval_entries) > 0:
    print(f"Example: {eval_entries[0]}")

In [ ]:
insert_evals_by_model_into_db(eval_entries)
print(f"{len(eval_entries)} eval entries inserted into db")